# Restart, Stop and Last

The aim of this notebook is to show how to continue or restart
a computation that was interrupted or finished. And explain how
to check the state of the sample during computation or stop it properly.


## Simple sample

First must have a sample to work with. Let us use the simple case of
the deposit of disks in box.

The disks are initially on a triangular lattice and there is no friction.

Since the input are already detailed in other notebook, only a simple
function is used to generate the DATBOX directory:

In [ ]:
import utils

nb_diskx = 1000
rmin = 1.5
rmax = 2.0
bodies = utils.generate(nb_diskx, rmin, rmax)

In [ ]:
from pylmgc90 import pre
pre.visuAvatars(bodies)

## Simple computation


#### Initialize and read

A simple computation is composed first by the common part
of a rigid script, which basically initialize the parameters
of the simulation and read the input data from the DATBOX directory :

In [ ]:
from pathlib import Path
import h5py
from pylmgc90 import chipy
from pylmgc90.chipy import computation

dim   = 2
mhyp  = 1
dt    = 5e-3
theta = 0.5

h5_file = 'lmgc90.h5'

In [ ]:
chipy.Initialize()
chipy.checkDirectories()
chipy.utilities_DisableLogMes()

# Set space dimension
chipy.SetDimension(dim,mhyp)
#
chipy.TimeEvolution_SetTimeStep(dt)
chipy.Integrator_InitTheta(theta)

chipy.ReadDatbox(deformable=False)

chipy.InitHDF5(h5_file)

chipy.OpenDisplayFiles()
chipy.OpenPostproFiles()

#### Computation

Then a time loop to run the computation. For the example's sake, let us imagine
that we did not use enough time steps to let all particles attain a rest state.

And to reduce the execution time and the storage space used, some low frequency
of file writing will be used:

In [ ]:
nb_steps     = 250
freq_write   =  10
freq_display =  25

dim = 2
theta = 0.5
stype = 'Stored_Delassus_Loops       '
norm  = 'Quad '
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10

In [ ]:
chipy.ComputeMass()
for k in range(1, nb_steps+1):
    if k%freq_display==0:
        print(f"computed step {k}")
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, freq_display)

computation.finalize()
print('Computation finished')

#### First results

Then by looking at the paraview file, it is seen that the deposit is not finished.
Thus the computation should be continued.

Furthermore, according to the number of steps computed and the frequency of file writing
provided, the number of record in the hdf5 file can be checked:

In [ ]:
with h5py.File(h5_file) as hf:
    nb_record = hf['Simulation/nb_record'][()]
    nb_record = int(nb_record)
print(f"Number of record in {h5_file}: {nb_record} = {nb_steps}/{freq_write}")

And the number of paraview files can also be counted:

In [ ]:
# !ls DISPLAY/*.vtmb | wc -l
display = Path('DISPLAY')
nb_display = sum(1 for _ in display.glob('*.vtmb'))
print(f"Number of display files : {nb_display} = {nb_steps}/{freq_display}")

## Restart computation

Now it is desired to continue the computation for another round to, hopefully,
finish the deposit.

To this end, the first important point is to specify to
read a new initial state after the `ReadDatbox` call with the `ReadIni` function
which take a record number as an input argument.

The second point is to make sure to not erase record in the hdf5 file and or display files.
* Regarding HDF5 files:
  * With HDF5, if an existing file is provided to the `InitHDF5` function, then all records with
    a `TPS` (which is the simulation time of the record) after the result of `TimeEvolution_GetTime`
    will be automatically erased.
  * a different input and output HDF5 file can be provided
  * if the number of record `-1` is used when reading state, then the last available record is read
* Regarding paraview files:
  * by default the numbering of the paraview files starts at 1
  * if a file with the same numbering already exists at time of writing, it is overwritten
  * that's why it is possible to provide a starting index for paraview files
In the end, to continue the computation and have the same results as if the computation
would be done in one go, the initialization step is:

In [ ]:
in_file = 'lmgc90.h5'
h5_file = 'lmgc90.h5'

chipy.Initialize()  
chipy.checkDirectories()  
chipy.utilities_DisableLogMes()

# Set space dimension  
chipy.SetDimension(dim,mhyp)
#  
chipy.TimeEvolution_SetTimeStep(dt)  
chipy.Integrator_InitTheta(theta)     
#               

chipy.ReadDatbox(deformable=False)
chipy.ReadIni(record=nb_record, h5_file=in_file)    
 
chipy.InitHDF5(h5_file)  

chipy.OpenDisplayFiles(nb_display+1)
chipy.OpenPostproFiles()

And the computation loop is then strictly the same, then the number of records/files can
be checked again:

In [ ]:
chipy.ComputeMass()
for k in range(1, nb_steps+1):
    if k%freq_display==0:
        print(f"computed step {k+nb_steps}")
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, freq_display)

computation.finalize()
print('Computation finished')

In [ ]:
with h5py.File(h5_file) as hf:
    nb_record = hf['Simulation/nb_record'][()]
    nb_record = int(nb_record)

display = Path('DISPLAY')
nb_display = sum(1 for _ in display.glob('*.vtmb'))

print(f"Number of record in {h5_file}: {nb_record} = {2*nb_steps}/{freq_write}")
print(f"Number of display files : {nb_display} = {2*nb_steps}/{freq_display}")

## Stop computation

Another approach would be to choose an oversized number of time step (or a while loop) and check,
during computation, if the deposit is stabilized and only then manually stop the computation.

In this special case it could be decided that there is no need for any paraview file
writing and that only the last step of the computation must be saved thus the frequency
of file writing will be set so that the script skips these steps.

To check/control the computation while it is running, this is possible thanks to the
`checkInteractiveCommand` function which is automatically included in the `compute_one_step` macro.
Depending on the existence of a file with an predefined name within the working directory,
some particular behaviour can be provoked (check the sphinx documentation for the exhaustive list).

To illustrate this, open a terminal or a file explorer window in the current directory
and be ready to create some files.

Then run the following loop and at some point of your choice, create the file `display_now`.
This should create a new paraview file at the end of the next computed step (and automatically
erase said file to not write again on next step).

When satisfied with the displayed state, you can then create the `stop` file to cleanly stop
the computation which will also write the last step in the output file.

In [ ]:
nb_steps = 100000000000
fwrite = fdisplay = nb_steps+1

h5_file = 'loop.h5'

dim   = 2
theta = 0.5
stype = 'Stored_Delassus_Loops       '
norm  = 'Quad '
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10
chipy.Initialize()  
chipy.checkDirectories()  
chipy.utilities_DisableLogMes()

# Set space dimension  
chipy.SetDimension(dim,mhyp)
#  
chipy.TimeEvolution_SetTimeStep(dt)  
chipy.Integrator_InitTheta(theta)     
#               

chipy.ReadDatbox(deformable=False)
 
chipy.InitHDF5(h5_file)  

chipy.OpenDisplayFiles()
chipy.OpenPostproFiles()

chipy.ComputeMass()
for k in range(1, nb_steps+1):
    if k%freq_display==0:
        print(f"computed step {k}")
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, fwrite, fdisplay)

computation.finalize()
print('Computation finished')

In [ ]:
with h5py.File(h5_file) as hf:
    nb_record = hf['Simulation/nb_record'][()]
    nb_record = int(nb_record)

display = Path('DISPLAY')
nb_display = sum(1 for _ in display.glob('*.vtmb'))

print(f"Number of record in {h5_file}: {nb_record}")
print(f"Number of display files : {nb_display}")

## Last

But not least ! This section is to explaine the concept of *Last* file.

In case of many number of time step with few record written, it may be wanted to save, at each
time step, the state of the computation but without keeping all of them.

This is when it is needed to add the `WriteLast` function :

In [ ]:
h5_file = 'lmgc.h5'
h5_last = 'last.h5'

nb_steps     = 500
freq_write   = 250
freq_display = nb_steps

dim   = 2
theta = 0.5
stype = 'Stored_Delassus_Loops       '
norm  = 'Quad '
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10
chipy.Initialize()
chipy.checkDirectories()
chipy.utilities_DisableLogMes()

# Set space dimension
chipy.SetDimension(dim,mhyp)
#
chipy.TimeEvolution_SetTimeStep(dt)
chipy.Integrator_InitTheta(theta)

chipy.ReadDatbox(deformable=False)

chipy.InitHDF5(h5_file)  

chipy.OpenDisplayFiles()
chipy.OpenPostproFiles()

chipy.ComputeMass()
for k in range(1, nb_steps+1):
    if k%freq_write==0:
        print(f"computed step {k}")
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, freq_display)
    chipy.WriteLast(h5_last)

computation.finalize()
print('Computation finished')